In [1]:
import torch, csv, emoji
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from transformers import AutoModel, AutoTokenizer 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer

from NN import FILEPATH, read_csv, emojiDataset, toEmoji

Using cpu device


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
a = read_csv(FILEPATH)
print(a[0][-1], a[1][-1])

In [ ]:
torch.cuda.is_available()

In [ ]:
dataset = emojiDataset()

In [ ]:
data_loader = DataLoader(dataset=dataset, batch_size=16, shuffle=True,num_workers=2)
sentences, emojis, labels_classes = dataset.get_others()

# data_iter = iter(data_loader)
# next_data = data_iter.next()
# features, labels = next_data
# print(features, labels)

toemoji = toEmoji()
learn_rate = optim.Adam(toemoji.parameters(), lr=0.005)
loss_func = nn.MSELoss()
# loss_func = nn.NLLLoss()
epochs = 10

In [ ]:
for i in range(epochs):
    for data in data_loader:
        features, labels = data
        toemoji.zero_grad()
        results = toemoji(features.float())
        # print(result)
        # print(results, labels)
        loss = loss_func(results, labels)
        # for result, label in zip(results, labels):
        #     loss += F.nll_loss(result, label) 
        loss.backward()
        learn_rate.step()
    print(loss)

In [ ]:
K = 2

with torch.no_grad():
    for i in range(len(sentences)) :
        feature, label = dataset[i]
        result = toemoji(feature.float())
        print(f'Sentence: {sentences[i]}\nOriginal labels: {emojis[i][0]}, output labels: {result[0]}\n') 
        # original_str, result_str = '', ''

        # top_k = (-result).argsort()[:K]
        # for top_index in top_k:
        #     result_str += labels_classes[top_index]
        
        # original_str = ''.join(emojis[i])
        
        # print(f'Sentence: {sentences[i]}\nOriginal labels: {original_str}, output labels: {result_str}\n')
        